In [1]:
import os
# import gspread_pandas
# import gspread
# import gspread_dataframe as gd
from google.api_core.retry import Retry
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from oauth2client.service_account import ServiceAccountCredentials
# from google.auth.transport.requests import AuthorizedSession
import pandas as pd

In [2]:
cred_path = "C:/Users/admin/Desktop/voice_data_queries/samboss-reward-firebase-adminsdk-9j6t2-097005da23.json" 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = cred_path    # os.environ은 환경 변수를 다루는 데 사용되는 Python의 내장 객체 

# Firebase initialize
try:
    firebase_admin.get_app()
except ValueError:
    cred = credentials.Certificate(cred_path)
    firebase_admin.initialize_app(cred)
    print("firebase_admin 권한 확인 완료.")

firebase_admin 권한 확인 완료.


In [22]:
db = firestore.Client()

users_ref = db.collection("Users").order_by("__name__")

# # Users의 모든 documents들을 가져옴
# users_docs = users_ref.stream()

# doc 수 카운트
total_doc_count = 0
with_attend_count = 0
without_attend_count = 0

# Get the first batch of documents
batch_size = 1000
users_docs = users_ref.limit(batch_size).stream()

while True:
    doc_count_in_batch = 0
    last_doc_snapshot = None
    for doc in users_docs:
        doc_count_in_batch += 1
        last_doc_snapshot = doc
        
        total_doc_count += 1
        data = doc.to_dict()
        subcollections = list(doc.reference.collections())
        has_attend_collection = any(coll.id == "Attend" for coll in subcollections)
        
        if has_attend_collection:
            with_attend_count += 1
        else:
            without_attend_count += 1

    if doc_count_in_batch < batch_size:
        # If the number of documents in the batch is less than the batch size, it means we've reached the end
        break
    
    # Get the next batch of documents starting after the last document in the current batch
    users_docs = users_ref.start_after(last_doc_snapshot).limit(batch_size).stream()

print(f"total_doc_count: {total_doc_count}, with_attend_count: {with_attend_count}, without_attend_count: {without_attend_count}")

total_doc_count: 33783, with_attend_count: 26041, without_attend_count: 7742


There are 0 documents in the 'Users' ollection.
There are 0 documents with an 'Attend' collection in the 'Users' collection.
There are 0 documents without an 'Attend' collection in the 'Users' collection.
